# AiFlix: Autonomous AI Filmmaking Workflow

This notebook demonstrates the end-to-end architecture of AiFlix. It orchestrates specialized AI agents to go from a high-level concept to a generated video sequence.

## Components:
1. **RAG Knowledge Base**: Retrieves cinematic theory (Syd Field, Storaro).
2. **Screenplay Agent**: Writes the script (DeepSeek-R1 logic).
3. **DOP Agent**: Creates a technical shot list (Qwen 3 logic).
4. **Media Engine**: Generates assets (FLUX.1 / CogVideoX).
5. **Assembly**: Edits the final cut (MoviePy).

In [ ]:
import sys
import os
from pathlib import Path

# Ensure repo root is in path
repo_root = Path(os.getcwd()).parent
if str(repo_root) not in sys.path:
    sys.path.append(str(repo_root))

from src.config import Config
from src.agents.orchestrator import Orchestrator
from src.media.image_gen import ImageGenerator
from src.media.video_gen import VideoGenerator
from src.media.editor import Editor

print("AiFlix modules loaded successfully.")

## Phase 1: Agent Orchestration
We initialize the Orchestrator with our `MockLLM` (or `OpenAI` if configured). The orchestrator will manage the Screenwriter and DOP agents.

In [ ]:
# Initialize Orchestrator
orchestrator = Orchestrator(llm_provider="mock")

# Define Concept
concept = "A cyberpunk detective investigating a neon-lit alleyway in Neo-Tokyo, discovering a glitching cat."

# Run Pipeline
result = orchestrator.run_pipeline(concept)
print("Pipeline execution complete.")

### Review Generated Artifacts
Let's look at the script and shot list generated by the agents.

In [ ]:
import json
print(json.dumps(result, indent=2))

## Phase 2: Media Generation
Now we iterate through the shot list and generate the actual video assets.

In [ ]:
image_gen = ImageGenerator()
video_gen = VideoGenerator()
editor = Editor()

video_clips = []
shot_list = result.get("shot_list", {}).get("shots", []) or result.get("shot_list", []) # Handle potential mock data format

# Safety check for mock data structure
if isinstance(shot_list, list) and len(shot_list) > 0:
    for i, shot in enumerate(shot_list):
        prompt = shot.get("visual_prompt", "Default prompt")
        shot_id = shot.get("id", i)
        
        # 1. Generate Image (Keyframe)
        img_path = Config.OUTPUT_DIR / f"shot_{shot_id}.png"
        image_gen.generate(prompt, img_path)
        
        # 2. Generate Video (Image-to-Video)
        vid_path = Config.OUTPUT_DIR / f"shot_{shot_id}.mp4"
        video_gen.generate(img_path, prompt, vid_path)
        
        video_clips.append(vid_path)
else:
    print("No shots found to generate.")

## Phase 3: Assembly
Finally, we stitch the clips together.

In [ ]:
final_movie_path = Config.OUTPUT_DIR / "final_movie.mp4"
if video_clips:
    editor.assemble(video_clips, final_movie_path)
    print(f"Movie saved to: {final_movie_path}")
else:
    print("Skipping assembly (no clips).")